## first make sure you have initiate the bert-server by open a terminal and type in the below

In [ ]:
#bert-server-start -model_dir ./multi_cased_L-12_H-768_A-12/ -num_worker=1

In [12]:
from bert_serving.client import BertClient
bc = BertClient()
out=bc.encode(['hur kan jag får support'])
out.shape

(1, 768)

In [13]:
bc = BertClient( show_server_config=False)
# encode a list of strings
with open('README.md') as fp:
    data = [v for v in fp if v.strip()][:512]
    num_tokens = sum(len([vv for vv in v.split() if vv.strip()]) for v in data)

show_tokens = True
bc.encode(data)  # warm-up GPU
for j in range(10):
    tmp = data * (2 ** j)
    c_num_tokens = num_tokens * (2 ** j)
    start_t = time.time()
    bc.encode(tmp, show_tokens=show_tokens)
    time_t = time.time() - start_t
    print('encoding %10d sentences\t%.2fs\t%4d samples/s\t%6d tokens/s' %
          (len(tmp), time_t, int(len(tmp) / time_t), int(c_num_tokens / time_t)))

server config:
                        client	=	1934b3e7-ca0f-42ab-94ef-f8dd01a2017b
                   num_process	=	2                             
          ventilator -> worker	=	['tcp://127.0.0.1:59235', 'tcp://127.0.0.1:59236', 'tcp://127.0.0.1:59237', 'tcp://127.0.0.1:59238', 'tcp://127.0.0.1:59239', 'tcp://127.0.0.1:59240', 'tcp://127.0.0.1:59241', 'tcp://127.0.0.1:59242']
                worker -> sink	=	tcp://127.0.0.1:59257         
           ventilator <-> sink	=	tcp://127.0.0.1:59234         
           server_current_time	=	2019-06-11 19:49:31.173275    
                     statistic	=	{'num_data_request': 4, 'num_total_seq': 6, 'num_sys_request': 6, 'num_total_request': 10, 'num_total_client': 6, 'num_active_client': 2, 'avg_request_per_client': 1.6666666666666667, 'min_request_per_client': 1, 'max_request_per_client': 2, 'num_min_request_per_client': 2, 'num_max_request_per_client': 4, 'avg_size_per_request': 2.0, 'min_size_per_request': 1, 'max_size_per_request': 3, '

C:\Users\zecharpy\AppData\Local\Continuum\anaconda3\envs\bert_py36\lib\site-packages\bert_serving\client\__init__.py:286: UserWarning: some of your sentences have more tokens than "max_seq_len=25" set on the server, as consequence you may get less-accurate or truncated embeddings.
here is what you can do:
- disable the length-check by create a new "BertClient(check_length=False)" when you do not want to display this warning
- or, start a new server with a larger "max_seq_len"
  '- or, start a new server with a larger "max_seq_len"' % self.length_limit)
C:\Users\zecharpy\AppData\Local\Continuum\anaconda3\envs\bert_py36\lib\site-packages\bert_serving\client\__init__.py:295: UserWarning: "show_tokens=True", but the server does not support showing tokenization info to clients.
here is what you can do:
- start a new server with "bert-serving-start -show_tokens_to_client ..."
- or, use "encode(show_tokens=False)"
  warnings.warn('"show_tokens=True", but the server does not support showing to

encoding         18 sentences	0.65s	  27 samples/s	   706 tokens/s
encoding         36 sentences	1.28s	  28 samples/s	   718 tokens/s
encoding         72 sentences	2.40s	  30 samples/s	   768 tokens/s
encoding        144 sentences	4.40s	  32 samples/s	   838 tokens/s
encoding        288 sentences	9.08s	  31 samples/s	   811 tokens/s
encoding        576 sentences	21.18s	  27 samples/s	   696 tokens/s
encoding       1152 sentences	51.07s	  22 samples/s	   577 tokens/s
encoding       2304 sentences	78.77s	  29 samples/s	   749 tokens/s
encoding       4608 sentences	157.76s	  29 samples/s	   748 tokens/s
encoding       9216 sentences	323.72s	  28 samples/s	   729 tokens/s


## test the dictionary for QnA maker

In [131]:

import numpy as np
from bert_serving.client import BertClient
from termcolor import colored

def whatisthis(s):
        if isinstance(s, str):
            #print("ordinary string")
            s=s.replace('Ã¥','å')
            s=s.replace('Ã¶','ö')
            s=s.replace('Ã¤','ä')
            #print(s)
            return s

        elif isinstance(s, unicode):
            print("unicode string")
            return s
        else:
            print("not a string")
            return s
def qna_dictionary(prefix_q='Q:',prefix_a='A:',file='qna.txt'):
    prefix_q =prefix_q
    prefix_a =prefix_a
    topk = 1
    indexer=0
    qna={}
    with open(file, encoding='utf-8-sig') as fp:
        lines = fp.readlines()
        lines=[line for line in lines if len(line)>1]
        cnt=0
        for l in lines:

            #print('{}'.format(str(cnt)),l)
            if l.startswith(prefix_q):
                #print(l)
                question=l.replace(prefix_q, '').strip()
                question=whatisthis(question)
                #print("Q",question)
                answer=lines[cnt+1].replace(prefix_a, '').strip()
                answer=whatisthis(answer)
                #print("A",answer)
                qna[question]=answer
                #print(cnt,l)
            cnt+=1
        return qna

    
t=qna_dictionary()

t.items()

dict_items([('Hur aktiverar jag mina Office-produkter?', 'Klicka på länkarna nedan för mer information om aktivering av Office-produkter'), ('Vad händer när nedladdningen är slutförd?', 'När nedladdningen slutförts går du till platsen där du sparade filen och dubbelklickar på filikonen för att starta installationen.'), ('Hur får jag support?', 'ring 020 120 32 05'), ('Vad är en ”incident”?', 'En "incident" är ett enskilt supportproblem och de rimliga åtgärder som krävs för att lösa problemet.'), ('Kan man använda mobiler ihop med Teams?', 'Självklart! Du kan via Teamsklienten få det att ringa ut på mobiltelefonen.'), ('Kan man spela in samtal?', 'Ja, samtalsinspelning kan göras med hjälp av tredjepartsapplikation för externa samtal.'), ('Växelfunktioner?', 'Ja självklart.'), ('Kan man integrera sin mail i Teams?', 'Det finns idag en integration via ”butiken” i Teams för e-post men inget officiellt tillägg från Microsoft.'), ('Hur starta man en gruppchatt', 'Välj ny chatt i Teams längst

In [132]:
%%writefile qna_example.py 
import numpy as np
from bert_serving.client import BertClient
from termcolor import colored

prefix_q ='Q:' 
answer ='A:'
topk = 1

def whatisthis(s):
        if isinstance(s, str):
            #print("ordinary string")
            s=s.replace('Ã¥','å')
            s=s.replace('Ã¶','ö')
            s=s.replace('Ã¤','ä')
            #print(s)
            return s

        elif isinstance(s, unicode):
            print("unicode string")
            return s
        else:
            print("not a string")
            return s
def qna_dictionary(prefix_q='Q:',prefix_a='A:',file='qna.txt'):
    prefix_q =prefix_q
    prefix_a =prefix_a
    topk = 1
    indexer=0
    qna={}
    with open(file, encoding='utf-8-sig') as fp:
        lines = fp.readlines()
        lines=[line for line in lines if len(line)>1]
        cnt=0
        for l in lines:

            #print('{}'.format(str(cnt)),l)
            if l.startswith(prefix_q):
                #print(l)
                question=l.replace(prefix_q, '').strip()
                question=whatisthis(question)
                print("Q",question)
                answer=lines[cnt+1].replace(prefix_a, '').strip()
                answer=whatisthis(answer)
                print("A",answer)
                qna[question]=answer
                #print(cnt,l)
            cnt+=1
        return qna
qna=qna_dictionary()
with open('qna.txt', encoding='utf-8-sig') as fp:
    questions = [v.replace(prefix_q, '').strip() for v in fp if v.strip() and v.startswith(prefix_q)]
    print('%d questions loaded, avg. len of %d' % (len(questions), np.mean([len(d.split()) for d in questions])))

with BertClient() as bc:
    doc_vecs = bc.encode(questions)

    while True:
        query = input(colored('din fråga: ', 'green'))
        query_vec = bc.encode([query])[0]
        # compute normalized dot product as score
        score = np.sum(query_vec * doc_vecs, axis=1) / np.linalg.norm(doc_vecs, axis=1)
        topk_idx = np.argsort(score)[::-1][:topk]
        #print('top %d questions similar to your query \n"%s"' % (topk, colored(query, 'green')))
        #print('top %d answer to your question %s')%(topk,colored(qna[whatisthis(query)],'yellow'))

        for idx in topk_idx:
            print('most similar question in our QnA list is \n---> %s\t%s' % (colored('%.1f' % score[idx], 'cyan'), colored(questions[idx], 'yellow')))
            q=whatisthis(questions[idx])
            
            print('answer from QnA maker txt \n ---> %s' % (qna[q]))

Overwriting qna_example.py


hur får jag support
